In [ ]:
!pip install pyngrok

In [ ]:
import requests
from flask import Flask, request, jsonify
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from pyngrok import ngrok

ngrok.set_auth_token("2oT2P0Bs49pEItaAxGFrrqWHv49_fU7QgsJQGWwaP4TX3JMv")

app = Flask(__name__)

# ngrok 터널 생성 및 외부 접속 URL 얻기
public_url = ngrok.connect(5000)
print(f"ngrok URL: {public_url}")

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("noahkim/KoT5_news_summarization")
model = AutoModelForSeq2SeqLM.from_pretrained("noahkim/KoT5_news_summarization")

def get_news_summary(query: str, full_text: str) -> str:
    try:

        prompt = f"""
            당신은 금융 분야의 요약 전문가입니다.

            아래의 금융 관련 텍스트를 읽고, 다음 지침에 따라 요약해 주세요:
            1. 전체 내용을 50단어 이내로 요약한 'summary' 필드를 작성해 주세요.
            2. 핵심 내용을 5-10개 간결하게 정리해서 'key_points' 필드에 작성해 주세요.
            3. 각 key_points 내용은 한 문장으로, 20단어를 넘지 않도록 해주세요.
            4. 텍스트의 핵심 키워드와 관련된 단어나 개념을 10개 내외로 'tags' 필드에 추가해 주세요.
            5. 전문 용어가 있다면 간단히 설명을 덧붙여 주세요.
            6. 숫자나 통계가 있다면 반드시 포함시켜 주세요.
            7. 요약은 객관적이고 중립적인 톤을 유지해 주세요.
            8. 연관된 회사가 있으면 해당 회사와의 관계를 설명해 주세요.
            9. {query}가 들어간 문장을 우선적으로 분석해 주시오.

            응답은 반드시 아래의 JSON 형식을 따라 주세요:
            {{
                "summary": "전체 내용 요약 (50단어 이내)",
                "key_points": [
                    "핵심 포인트 1 (20단어 이내)",
                    "핵심 포인트 2 (20단어 이내)",
                    "핵심 포인트 3 (20단어 이내)",
                    "핵심 포인트 4 (20단어 이내)",
                    "핵심 포인트 5 (20단어 이내)",
                    ...
                ],
                "tags": ["태그1", "태그2", "태그3", ...]
            }}

            텍스트:
            {full_text}

            위 지침에 따라 JSON 형식으로 요약해 주세요.
            """

        inputs = tokenizer(prompt, return_tensors="pt", max_length=3400, truncation=True)
        summary_ids = model.generate(inputs['input_ids'], max_length=1500, num_beams=4, early_stopping=True)
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

        # # tokenizer로 입력을 생성하고, 'input_ids'를 MPS 디바이스로 이동
        # inputs = tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True).to(device)
        # print(inputs)
        # # inputs = {key: value.to(device) for key, value in inputs.items()}
        # #print(inputs)

        # 모델 생성
        summary_ids = model.generate(inputs['input_ids'], max_length=1024, num_beams=4, early_stopping=True)
        # 생성된 텍스트 디코딩
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

        return summary
    except Exception as e:
        print(f"Failed to generate summary: {e}")
        return ""

@app.route('/data', methods=['POST'])
def receive_data():
    try:
        # POST 요청에서 JSON 데이터를 받습니다.
        data = request.json

        # 필수 데이터가 있는지 확인
        if not data or not data.get('query') or not data.get('full_text'):
            return jsonify({"error": "Missing 'query' or 'full_text' in request data"}), 400

        query = data.get('query')
        full_text = data.get('full_text')
        print(f"Received data: query = {query}, full_text = {full_text}")

        # 요약 생성 함수 호출 (예시: transformers 모델 사용)
        summary = get_news_summary(query, full_text)
        print(summary)
        if not summary:
            return jsonify({"error": "Failed to generate summary."}), 500

        return jsonify({"summary": summary})

    except Exception as e:
        print(f"Error: {e}")
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
    # ngrok URL을 가져오기 전에 ngrok이 실행되도록 기다림
    # ngrok_url = get_ngrok_url()
    app.run(debug=True, use_reloader=False)

ngrok URL: NgrokTunnel: "https://4eff-34-143-207-67.ngrok-free.app" -> "http://localhost:5000"


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


Received data: query = AMD, full_text = “테무 계속 써도 되는거야?”…실적부진에 주가 29% 급락 [투자360]
게티이미지·게티이미지뱅크

'엔비디아 대항마' AMD, 서버 제조업체 ZT 시스템 인수
지난달엔 AI 스타트업 인수…AI 칩 경쟁 가속·주가 3% 상승




...

금융 분야의 요약 전문가인 당신은 아래의 금융 관련 텍스트를 읽고 다음 지침에 따라 요약해 줄 것이며                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

INFO:werkzeug:127.0.0.1 - - [09/Nov/2024 15:42:59] "POST /data HTTP/1.1" 200 -


뉴욕증시에서 주요 주가지수가 또 동반 하락했는데 기술주에서 우량주로 순환매한다는 분석이 무색할 만큼 업종을 가리지 않고 전반적으로 낙폭이 가팔랐으며                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

INFO:werkzeug:127.0.0.1 - - [09/Nov/2024 15:44:38] "POST /data HTTP/1.1" 200 -


인공지능 선두주자 엔비디아의 주가가 4거래일 연속 상승세를 멈추고 반락, 원인에 관심이 쏠린 가운데,                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [ ]:
!pip install cohere pyngrok

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import requests
from flask import Flask, request, jsonify
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from pyngrok import ngrok
import cohere
import re
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer,RobertaForSequenceClassification
import torch
#from transformers import pipeline
import numpy as np

co = cohere.Client('')
ngrok.set_auth_token('')

class args:
    # ---- factor ---- #
    debug=False
    amp = True
    gpu = '0'

    epochs=5
    batch_size=32
    weight_decay=1e-6
    n_fold=5
    fold=3 # [0, 1, 2, 3, 4] # 원래는 3

    exp_name = 'model_f'
    dir_ = f'./saved_models/'
    pt = 'mz_model'
    max_len = 128

    start_lr = 1e-5#1e-3,5e-5
    min_lr=1e-6
    # ---- Dataset ---- #

    # ---- Else ---- #
    num_workers=8
    seed=2021
    scheduler = None

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base')
model = RobertaForSequenceClassification.from_pretrained('klue/roberta-base', num_labels = 3)
model_path = "path/to/klue_base_fold3_s.pth"

state_dict = torch.load(model_path, map_location=device)
model.load_state_dict(state_dict)
model.to(device)
model.eval()


app = Flask(__name__)

def predict_label(sentence):
    inputs = tokenizer(
        sentence,
        return_tensors="pt",
        max_length=args.max_len,
        truncation=True,
        padding="max_length"
    )

    input_ids = inputs['input_ids'].to(device)
    token_type_ids = inputs['token_type_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)
        logits=outputs.logits
        predicted_label = logits.argmax(dim=1).item()

    return predicted_label

# ngrok 터널 생성 및 외부 접속 URL 얻기
public_url = ngrok.connect(5000)
print(f"ngrok URL: {public_url}")

def get_news_summary(query: str, full_text: str) -> str:
    try:
        response = co.summarize(
            text=full_text,
            additional_command="output should be translated into korean",
        )
        summary = response.summary

        if isinstance(summary, list):
            summary = ' '.join(summary)

        return summary
    except Exception as e:
        print(f"Failed to generate summary: {e}")
        return ""


@app.route('/data', methods=['POST'])
def receive_data():
    try:
        data = request.json
        print(data)

        if not data or not data.get('query') or not data.get('full_text'):
            return jsonify({"error": "Missing 'query' or 'full_text' in request data"}), 400

        query = data.get('query')
        full_text = data.get('full_text')
        print(f"Received data: query = {query}, full_text = {full_text}")

        summary = get_news_summary(query, full_text)
        if not summary:
            return jsonify({"error": "Failed to generate summary."}), 500

        label = predict_label(summary)
        print(f"Translated text: {summary}, Predicted label: {label}")

        return jsonify({"summary": summary, "label": label})

    except Exception as e:
        print(f"Error: {e}")
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-10-28ffabbcd36a>:46: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly a

ngrok URL: NgrokTunnel: "https://d30a-35-231-126-171.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


{'query': 'NVDA', 'full_text': '美 반도체 인텔, 2분기 매출 호조 전망에 주가 6% 급등 6아직도 문신 지우는 나나 "없애는 게 5배 아파"\n\n美 뉴욕증시, 국가신용등급 강등·NVDA 주가급등 속 혼조마감 다우 0.11%↓…S&P 0.88%·나스닥 1.71%↑ 뉴욕증권거래소의 트레이더들. ⓒ로이터/연합뉴스 미국 뉴욕증시가 미국 국가 신용 등급 강등 경고와 엔비디아(NVDA)의 주가 급등에 혼조세로 마감했다.   25일(현지시간) 뉴욕증권거래소(NYSE)에서 다우존스30산업 지수는 전장보다 35.27포인트(0.11%) 내린 3만2764.65에 거래를 마쳤다.   반면 스탠더드앤드푸어스(S&P)500지수는 전장 대비 36.04포인트(0.88%) 오른 4151.28을 기록했다. 기술주 중심의 나스닥 종합지수는 213.93포인트(1.71%) 상승한 1만2698.09에 장을 마감했다.   증시는 이날 신용평가사 피치의 미국 국가 신용 등급 강등에도 엔비디아의 급등세에 힘입어 기술주들이 상승했다.   미국의 채무불이행(디폴트)을 둘러싼 부채 한도 협상에 계속 차질이 빚어지는 가운데, 글로벌 신용평가사 피치는 전날 미국의 국가신용등급을 부정적 관찰 대상으로 지정하며 강등 가능성을 경고했다.   시장을 이끈 것은 전날 장 마감 뒤 예상을 훌쩍 뛰어넘는 실적을 발표한 엔디비아다. 엔비디아 주가는 이날 무려 24.37% 폭등했다. 인공지능(AI)에 사용되는 엔비디아 반도체에 주가가 오르고 있다. 엔비디아 주가는 미국 주식 역사상 하루 기준 최대 상승폭을 보였다고 파이낸셜타임스(FT)는 평가했다   주요 6개국 통화에 대한 달러화 가치를 보여주는 달러인덱스 지수는 전장보다 0.37% 올랐다.   미 뉴욕상업거래소(NYMEX)에서 7월 인도분 서부텍사스산원유(WTI)는 전 거래일보다 2.51달러(3.38%) 하락한 배럴당 71.83달러에 거래를 마쳤다.\n\n'}
Received data: query = NVDA, full_text = 美 반도체 인텔

INFO:werkzeug:127.0.0.1 - - [10/Nov/2024 14:28:14] "POST /data HTTP/1.1" 200 -


...

Translated text: Kim Min-jae, a defenseman for the NHL's New York Rangers, has been selected as a candidate for the best goalkeeper in the Italian league "Serie A" for this season. Kim Min-jae is the first Asian to be nominated for the award, which was first established in 2018. The award has been won by Italian goalkeeper Gianluigi Donnarumma, French goalkeeper Mike Maignan and British goalkeeper Ederson Moraes in the past. If Kim Min-jae wins the award this time, he will be the first Asian to win the award. In other news, officials from the Financial Supervisory Service's Securities and Futures Commission were arrested without notice for alleged violations of the Securities Exchange Act. The officials are alleged to have used insider information about the suspension of the activities of the boy band BTS and traded stocks in violation of the law prior to the public announcement. It is alleged that they avoided losses of KRW 2.3 billion won (USD 1.5 million). The Securities and Futures

INFO:werkzeug:127.0.0.1 - - [10/Nov/2024 14:32:12] "POST /data HTTP/1.1" 500 -


{'query': 'A', 'full_text': '걸려도 금방 풀려나…주가조작 넷중 한명은 재범, 한탕 부추긴다 전체 재판 20%는 사기·공갈 10명중 4명은 실형도 면해 전세사기 1년새 3000% 증가  주식시장 3대 불공정거래 약한 처벌에 재범률 23%  사회지도층 부패 반복되며 공동체 신뢰 무너진 상황 사진 확대 [사진=연합뉴스] “한국이 불신사회가 된 배경에는 지난 10년동안 사회 리더들이 보인 부패가 자리합니다. 앞에서는 ‘정의’를 외쳐놓고, 뒤에서는 반칙과 편법을 저질렀습니다. 힘 있고 돈 있는 자에게 솜방망이 처벌을 한 법을 더 이상 시민들이 신뢰하지 않게 된 이유입니다.” 국내 유명 사립대학교에서 15년 넘게 법학을 가르쳐 온 A 교수는 최근 매일경제와의 통화에서 대한민국이 ‘불신사회’로 떨어진 배경을 이렇게 설명했다. 권력자들이 앞장서 부정한 짓을 저지르고도 합당한 처벌이 이뤄지지 않으면서 서로 속고 속이는 불법행위가 만연해졌다는 진단이다. 그는 “바가지·전세사기·주가조작·보이스피싱 등 한국 사회의 신뢰를 붕괴시키는 행위들은 유형만 다를 뿐, 서로를 속이고 신뢰를 무너뜨리는 범죄라는 점에서 그 뿌리가 같다”고 말했다. A교수의 말처럼 올해는 유독 공동체 신뢰를 저해하는 범죄행위가 많았다. 이제 막 사회생활을 시작해 부동산 계약 관련 지식이 부족한 20~30대 젊은 세대를 겨냥한 전세사기, 개인 투자자를 속여 거액을 빼돌린 주가조작이 잇달아 적발되기도 했다. 범죄 행위까지는 아니더라도 재래시장과 지역축제에 만연한 바가지 역시 이슈가 되기도 했다. 경찰청이 24일 발표한 전세사기 특별단속 결과에 따르면 지난 1년간 3466명을 검거하고 367명을 구속했다. 몰수ㆍ추징보전 금액은 5억5000만원에서 172억원으로 증가했다. 증가율은 3040%였다. 지난 11일 서울시의회 국민의힘 최진혁 의원은 “지난해 8월부터 올해 5월까지 서울에서 10개월간 총 2709건의 전세사기가 발생했다”고 발표했다. 피해 금액은 6935억원으로 집계됐다. 사진 확대 ‘불

INFO:werkzeug:127.0.0.1 - - [10/Nov/2024 14:32:38] "POST /data HTTP/1.1" 200 -


Translated text: Hong Kong's Hang Seng index is expected to fall more than 8% in the first half of next year, which could lead to losses for investors with equity-linked securities (ELS) linked to the index. The Hang Seng is expected to fall to around 6,500 points in the first half of next year, down more than 8% from the end of this year, when the index is expected to fall to around 7,000 points. The total size of the ELS that banks sold for the Hang Seng index next year is estimated to be about 8 trillion won. Some experts say that if the Hang Seng index does not rise above 7,000 points in the first half of next year, most investors are likely to suffer losses. Due to the recent sharp rise in stock market risks, experts predict that the Hang Seng index could fall sharply in the first half of next year. Due to the recent sharp rise in stock market risks, experts predict that the Hang Seng index could fall sharply in the first half of next year. Experts predict that the Hang Seng index